In [34]:
import numpy as np
from sklearn import model_selection, datasets, svm
from qiskit import QuantumCircuit, Aer, QuantumRegister, ClassicalRegister, execute
import matplotlib.pyplot as plt
import copy

In [37]:
def initilialize_circuit(N):
    
    q = QuantumRegister(N)
    c = ClassicalRegister(1)
    qc = QuantumCircuit(q,c)
    
    return qc, c 

In [38]:
def feature_map(qc, X):
    
    for i, x in enumerate(X): qc.rx(x, i)
        
    return qc     

In [39]:
def variational_circuit(qc, theta, N):
    
    for i in range(N-1):
        qc.cx(i, i+1)
    
    qc.cx(N-1, 0)
    
    for i in range(N):
        qc.ry(theta[i], i)
        
    return qc     

In [40]:
def quantum_nn(X, theta, N, simulator=True, shots=1e4):
    qc,c = initilialize_circuit(N)
    qc = feature_map(qc, X)
    qc.barrier()
    qc = variational_circuit(qc, theta, N)
    qc.barrier()
    qc.measure(0,c)
    qc.draw('mpl')
    
    backend = Aer.get_backend('qasm_simulator')
    jobs = execute(qc, backend, shots=shots)
    result = jobs.result()
    counts = result.get_counts(qc)

    return counts['1']/shots

In [41]:
def loss_fn(pred, target): return (pred-target)**2

In [42]:
def gradient(X, Y, theta, N):
    delta=0.01
    grad=[]

    for i in range(len(theta)):
        dtheta = copy.copy(theta)
        dtheta[i] += delta

        pred1 = quantum_nn(X, dtheta, N)
        pred2 = quantum_nn(X, theta, N)

        grad.append((loss_fn(pred1, Y)-loss_fn(pred2, Y))/delta)

    return np.array(grad)

In [43]:
def accuracy(X, Y, theta, N):
    counter = 0

    for X_i, Y_i in zip(X, Y):
        prediction = quantum_nn(X_i, theta, N)

        if prediction < 0.5 and Y_i==0:
            counter += 1
        if prediction >= 0.5 and Y_i==1:
            counter += 1

    return counter/len(Y)

In [ ]:
iris = datasets.load_iris()

X = iris['data'][:100]
Y = iris['target'][:100]

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    X,
    Y,
    test_size=0.33,
    random_state=42
)

In [ ]:
n_qubits = X.shape[1]
print(f"Numer of Qubits = Number of Features: {n_qubits}")

In [44]:
lr = 0.05
loss_list = []
epochs = 12
theta = np.ones(n_qubits)

print('Epoch \t Loss \t Accuracy')

for i in range(epochs):
    
    loss_tmp = []
    
    for X_i, Y_i in zip(X_train, Y_train):
        pred = quantum_nn(X_i, theta, N = n_qubits)
        loss_tmp.append(loss_fn(pred, Y_i))
        theta = theta-lr*gradient(X_i, Y_i, theta, N=n_qubits)

    loss_list.append(np.mean(loss_tmp))
    acc = accuracy(X_train, Y_train, theta, N=n_qubits)

    print(f'{i} \t {loss_list[-1]:.3f} \t {acc:.3f}')
     

Epoch 	 Loss 	 Accuracy
0 	 0.269 	 0.776
1 	 0.228 	 0.866
2 	 0.225 	 0.836
3 	 0.231 	 0.821
4 	 0.218 	 0.866
5 	 0.217 	 0.836
6 	 0.213 	 0.806
7 	 0.215 	 0.791
8 	 0.209 	 0.851
9 	 0.207 	 0.806
10 	 0.208 	 0.791
11 	 0.213 	 0.806


In [45]:
accuracy(X_test, Y_test, theta, N = n_qubits)

0.7272727272727273